# 📗 발표용 Part 3: 모델링 및 최적화 v2

## 🎯 Part 3 목표 및 이전 Part 요약

### 이전 Part 완료 사항

**Part 1: 문제 정의 및 핵심 발견**
- 50,105개 한국 기업, 부도율 1.51% (1:66 불균형)
- **핵심 발견**: 유동성이 가장 강력한 예측 변수
- 업종별 부도율 2배 차이 (제조업 vs 서비스업)

**Part 2: 도메인 특성 공학**
- 52개 도메인 기반 특성 생성
- VIF/IV/AUC 기반 특성 선택 → **27개 최종 특성**
- 7개 카테고리: 유동성, 지급불능, 재무조작, 한국시장, 이해관계자, 복합리스크, 상호작용

### Part 3 v2 목표 및 개선 사항

**🚨 핵심 개선: Data Leakage 완전 제거**

```
❌ v1의 문제점:
- Test set으로 모델 선택
- Test set으로 임계값 최적화
- Test set으로 Traffic Light 기준 결정

✅ v2 해결책:
- 3-Way Split (Train/Validation/Test)
- Test set은 최종 평가 직전 단 한 번만
- 모든 의사결정은 Validation set 기반
```

**주요 개선 사항:**
1. **3-Way Data Split** (Train 60% / Validation 20% / Test 20%)
2. **리샘플링 전략 대조 실험** (SMOTE vs Class Weight)
3. **Validation 기반 모델 선택** + Statistical Significance Test
4. **Validation 기반 임계값 최적화** (F2-Score, Recall 우선)
5. **데이터 기반 Traffic Light** (Recall 80%/95% 보장)
6. **Test Set은 마지막 평가만** (절대 의사결정에 사용 안 함)

**목표 성능:**
- PR-AUC: 0.15~0.20 (불균형 데이터 고려)
- F2-Score: 0.35~0.50 (Recall 우선)
- Recall: 60~80%
- Type II Error: 20~40% (부도 미탐지 최소화)

---

## 0. 환경 설정

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import warnings
import platform
import os
import joblib
import pickle
from datetime import datetime

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 머신러닝 - 전처리
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# 머신러닝 - 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

# 머신러닝 - 튜닝
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline

# 불균형 데이터 처리
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN

# 평가 메트릭
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve,
    make_scorer, fbeta_score, recall_score, precision_score
)

# 통계 검정
from scipy.stats import wilcoxon, ttest_rel

# 경고 무시
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (CLAUDE.md 준수)
if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# 랜덤 시드 (재현성)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ 환경 설정 완료")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")
print(f"   - Scikit-learn: {sklearn.__version__}")
print(f"   - LightGBM: {lgb.__version__}")
print(f"   - XGBoost: {xgb.__version__}")
print(f"   - CatBoost: {catboost.__version__}")
print(f"   - Platform: {platform.system()}")
print(f"   - Random State: {RANDOM_STATE}")

In [ ]:
# sklearn 버전 추가 임포트
import sklearn
import catboost

## 1. 데이터 로딩 및 3-Way Split ⭐

### 1.1 데이터 로딩

Part 2에서 생성한 **27개 도메인 기반 특성**을 로딩합니다.

In [ ]:
# 데이터 경로 (하드코딩 금지)
DATA_DIR = '../data'
FEATURES_FILE = os.path.join(DATA_DIR, 'features', 'domain_based_features_완전판.csv')
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')

# 디렉토리 생성
os.makedirs(PROCESSED_DIR, exist_ok=True)

# 데이터 로딩
print(f"📂 데이터 로딩 중: {FEATURES_FILE}")
df = pd.read_csv(FEATURES_FILE, encoding='utf-8')

print(f"✅ 데이터 로딩 완료")
print(f"   - 전체 기업 수: {len(df):,}")
print(f"   - 전체 변수 수: {len(df.columns)}")
print(f"   - 메모리 사용: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

df.head(3)

In [ ]:
# 타겟 변수 확인
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'

print(f"🎯 타겟 변수: {TARGET_COL}")
print(f"\n부도 분포:")
print(df[TARGET_COL].value_counts())
print(f"\n부도율: {df[TARGET_COL].mean():.4%}")
print(f"불균형 비율: 1:{int(1/df[TARGET_COL].mean())}")

### 1.2 특성 및 타겟 분리

In [ ]:
# 특성(X)과 타겟(y) 분리
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

print(f"✅ 특성 및 타겟 분리 완료")
print(f"   - X shape: {X.shape}")
print(f"   - y shape: {y.shape}")
print(f"   - 특성 목록 (27개):")
for i, col in enumerate(X.columns, 1):
    print(f"      {i:2d}. {col}")

### 1.3 3-Way Split: Train / Validation / Test ⭐⭐⭐

**🚨 Data Leakage 방지를 위한 핵심 설계:**

```
전체 데이터 (50,105)
├─ Train Set (60%, ~30,063): 모델 학습 + CV 튜닝
├─ Validation Set (20%, ~10,021): 모델 선택, 임계값 최적화, 의사결정
└─ Test Set (20%, ~10,021): 최종 평가만 (절대 건드리지 않음!)
```

**중요:**
- Test set은 이 노트북 마지막 섹션에서 **단 한 번만** 평가
- 모든 의사결정(모델 선택, 임계값 등)은 **Validation set**에서만
- Stratified split으로 각 set의 부도율 동일하게 유지

In [ ]:
# 1차 분할: Train+Val (80%) vs Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=RANDOM_STATE
)

# 2차 분할: Train (75% of 80% = 60%) vs Validation (25% of 80% = 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=0.25, 
    stratify=y_temp, 
    random_state=RANDOM_STATE
)

print("="*70)
print("✅ 3-Way Split 완료")
print("="*70)
print(f"Train Set:      {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%, 부도율: {y_train.mean():.4%})")
print(f"Validation Set: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%, 부도율: {y_val.mean():.4%})")
print(f"Test Set:       {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%, 부도율: {y_test.mean():.4%})")
print("="*70)
print("⚠️  Test Set은 최종 평가 전까지 절대 사용하지 않음!")
print("="*70)

In [ ]:
# 데이터 분할 시각화
fig = go.Figure()

sets = ['Train', 'Validation', 'Test']
sizes = [len(X_train), len(X_val), len(X_test)]
bankruptcy_rates = [y_train.mean()*100, y_val.mean()*100, y_test.mean()*100]

fig.add_trace(go.Bar(
    x=sets,
    y=sizes,
    text=[f"{s:,}<br>({s/len(X)*100:.1f}%)" for s in sizes],
    textposition='auto',
    name='기업 수',
    marker_color=['#3498db', '#2ecc71', '#e74c3c']
))

fig.update_layout(
    title='3-Way Data Split (Train/Validation/Test)',
    xaxis_title='데이터셋',
    yaxis_title='기업 수',
    height=400
)

fig.show()

# 부도율 확인
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=sets,
    y=bankruptcy_rates,
    text=[f"{r:.4%}" for r in [r/100 for r in bankruptcy_rates]],
    textposition='auto',
    marker_color=['#3498db', '#2ecc71', '#e74c3c']
))

fig2.update_layout(
    title='부도율 분포 (Stratified Split 검증)',
    xaxis_title='데이터셋',
    yaxis_title='부도율 (%)',
    height=400
)

fig2.show()

print("✅ 각 데이터셋의 부도율이 동일하게 유지됨 (Stratified Split 성공)")

### 1.4 데이터 기본 통계

In [ ]:
# Train set 기본 통계
print("📊 Train Set 기본 통계 (전처리 전)")
print("="*70)

# 결측치 확인
missing_counts = X_train.isnull().sum()
missing_pct = (missing_counts / len(X_train) * 100).round(2)

if missing_counts.sum() > 0:
    print("\n⚠️ 결측치 발견:")
    missing_df = pd.DataFrame({
        '결측 수': missing_counts[missing_counts > 0],
        '비율(%)': missing_pct[missing_counts > 0]
    }).sort_values('비율(%)', ascending=False)
    print(missing_df)
else:
    print("✅ 결측치 없음")

# 무한대 값 확인
inf_counts = np.isinf(X_train).sum()
if inf_counts.sum() > 0:
    print("\n⚠️ 무한대 값 발견:")
    print(inf_counts[inf_counts > 0])
else:
    print("✅ 무한대 값 없음")

# 기본 통계량
print("\n📈 Train Set 기술 통계:")
print(X_train.describe().T[['mean', 'std', 'min', 'max']])

---

## ✅ Section 1 완료

**완료 사항:**
- ✅ 환경 설정 및 라이브러리 로딩
- ✅ 데이터 로딩 (50,105개 기업, 27개 특성)
- ✅ 3-Way Split (Train 60% / Validation 20% / Test 20%)
- ✅ Stratified Split 검증 (부도율 동일 유지)
- ✅ 데이터 기본 통계 확인

**다음 단계:**
- Section 2: 전처리 파이프라인 정의
- Section 3: 리샘플링 전략 대조 실험 (SMOTE vs Class Weight)

**⚠️ 중요 확인:**
- Test set은 아직 전혀 사용하지 않음 ✅
- 모든 작업은 Train/Validation set에서만 진행 ✅

## 2. 전처리 파이프라인 정의

### 2.1 Custom Transformers

**전처리 순서 (CLAUDE.md 준수):**
1. 무한대 값 처리 (InfiniteHandler)
2. 결측치 보간 (SimpleImputer - median)
3. 로그 변환 (LogTransformer - 선택적)
4. 스케일링 (RobustScaler - 이상치에 강건)

**참고**: Tree 기반 모델은 이상치에 강건하므로 Winsorizer는 제외

In [ ]:
# Custom Transformer 1: 무한대 값 처리
class InfiniteHandler(BaseEstimator, TransformerMixin):
    """무한대 값을 0으로 대체"""
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X = X.replace([np.inf, -np.inf], 0)
        return X

# 테스트
print("✅ InfiniteHandler 정의 완료")

### 2.2 파이프라인 생성 함수

In [ ]:
def create_preprocessing_pipeline(use_log=False):
    """
    전처리 파이프라인 생성
    
    Parameters:
    -----------
    use_log : bool, default=False
        로그 변환 적용 여부 (Tree 모델은 불필요)
    
    Returns:
    --------
    pipeline : Pipeline
    """
    steps = [
        ('inf_handler', InfiniteHandler()),
        ('imputer', SimpleImputer(strategy='median')),
    ]
    
    # Tree 모델에는 스케일링 불필요하지만, 앙상블에 LR 포함 가능성 고려
    steps.append(('scaler', RobustScaler()))
    
    return Pipeline(steps)

# 테스트
test_pipeline = create_preprocessing_pipeline()
print("✅ 전처리 파이프라인 생성 함수 정의 완료")
print(f"\n파이프라인 단계:")
for name, transformer in test_pipeline.steps:
    print(f"  - {name}: {transformer.__class__.__name__}")

## 3. 리샘플링 전략 대조 실험 ⭐

### 3.1 전략 설계

**문제**: SMOTE + Class Weight 동시 사용 시 노이즈 증폭, 과적합 위험

**해결**: 두 전략을 명확히 분리하여 실험

```
Strategy A: SMOTE 계열 (Class Weight 없음)
  - SMOTE(0.2)
  - BorderlineSMOTE(0.2)
  - SMOTETomek(0.2)

Strategy B: Class Weight Only (리샘플링 없음)
  - scale_pos_weight = sqrt(neg/pos)
  - scale_pos_weight = neg/pos
  - class_weight = 'balanced'
```

**평가**: Validation Set에서 PR-AUC, F2-Score 비교

**권장**: 금융 데이터 특성상 **Strategy B (Class Weight)**가 더 나을 가능성 높음

In [ ]:
# 불균형 비율 계산
n_pos = y_train.sum()
n_neg = len(y_train) - n_pos
imbalance_ratio = n_neg / n_pos

print(f"📊 Train Set 불균형 분석")
print(f"="*70)
print(f"부도 기업 (Positive):  {n_pos:,} ({n_pos/len(y_train)*100:.2f}%)")
print(f"정상 기업 (Negative):  {n_neg:,} ({n_neg/len(y_train)*100:.2f}%)")
print(f"불균형 비율:           1:{imbalance_ratio:.1f}")
print(f"="*70)
print(f"\nClass Weight 계산:")
print(f"  - sqrt(neg/pos) = {np.sqrt(imbalance_ratio):.2f}")
print(f"  - neg/pos = {imbalance_ratio:.2f}")

### 3.2 빠른 실험: LightGBM으로 두 전략 비교

전체 AutoML 전에 LightGBM 하나로 빠르게 두 전략을 비교합니다.

In [ ]:
# 전처리 (Train set에 fit, Val set에 transform)
preprocessor = create_preprocessing_pipeline()
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# DataFrame으로 변환 (컬럼명 유지)
X_train_processed = pd.DataFrame(X_train_processed, columns=X_train.columns, index=X_train.index)
X_val_processed = pd.DataFrame(X_val_processed, columns=X_val.columns, index=X_val.index)

print("✅ 전처리 완료 (Train set fit, Validation set transform)")
print(f"   - X_train_processed: {X_train_processed.shape}")
print(f"   - X_val_processed: {X_val_processed.shape}")

In [ ]:
# Strategy A: SMOTE 계열
print("📊 Strategy A: SMOTE 계열 테스트")
print("="*70)

smote_methods = {
    'SMOTE': SMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE),
    'BorderlineSMOTE': BorderlineSMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE),
    'SMOTETomek': SMOTETomek(sampling_strategy=0.2, random_state=RANDOM_STATE)
}

strategy_a_results = {}

for name, sampler in smote_methods.items():
    # 리샘플링
    X_resampled, y_resampled = sampler.fit_resample(X_train_processed, y_train)
    
    # LightGBM 학습 (Class Weight 없음)
    model = lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )
    
    model.fit(X_resampled, y_resampled)
    
    # Validation 평가
    y_val_prob = model.predict_proba(X_val_processed)[:, 1]
    pr_auc = average_precision_score(y_val, y_val_prob)
    
    # F2-Score 계산 (최적 임계값)
    precisions, recalls, thresholds = precision_recall_curve(y_val, y_val_prob)
    beta = 2
    f2_scores = (1 + beta**2) * (precisions * recalls) / (beta**2 * precisions + recalls + 1e-10)
    best_f2 = np.max(f2_scores)
    
    strategy_a_results[name] = {
        'pr_auc': pr_auc,
        'f2_score': best_f2,
        'resampled_size': len(X_resampled),
        'resampled_pos_rate': y_resampled.mean()
    }
    
    print(f"{name:20s} | PR-AUC: {pr_auc:.4f} | F2-Score: {best_f2:.4f} | "
          f"Resampled: {len(X_resampled):,} (부도율: {y_resampled.mean():.2%})")

print("="*70)

In [ ]:
# Strategy B: Class Weight Only
print("\n📊 Strategy B: Class Weight Only 테스트")
print("="*70)

class_weight_methods = {
    'No Weight': 1,
    'Sqrt Weight': np.sqrt(imbalance_ratio),
    'Full Weight': imbalance_ratio
}

strategy_b_results = {}

for name, weight in class_weight_methods.items():
    # LightGBM 학습 (리샘플링 없음)
    model = lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,
        scale_pos_weight=weight,  # Class Weight
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )
    
    model.fit(X_train_processed, y_train)
    
    # Validation 평가
    y_val_prob = model.predict_proba(X_val_processed)[:, 1]
    pr_auc = average_precision_score(y_val, y_val_prob)
    
    # F2-Score 계산
    precisions, recalls, thresholds = precision_recall_curve(y_val, y_val_prob)
    beta = 2
    f2_scores = (1 + beta**2) * (precisions * recalls) / (beta**2 * precisions + recalls + 1e-10)
    best_f2 = np.max(f2_scores)
    
    strategy_b_results[name] = {
        'pr_auc': pr_auc,
        'f2_score': best_f2,
        'weight': weight
    }
    
    print(f"{name:20s} | PR-AUC: {pr_auc:.4f} | F2-Score: {best_f2:.4f} | Weight: {weight:.2f}")

print("="*70)

In [ ]:
# 결과 비교
print("\n📊 리샘플링 전략 비교 결과 (Validation Set 기반)")
print("="*70)

# Strategy A 최고 성능
best_a_name = max(strategy_a_results, key=lambda k: strategy_a_results[k]['pr_auc'])
best_a_pr_auc = strategy_a_results[best_a_name]['pr_auc']

# Strategy B 최고 성능
best_b_name = max(strategy_b_results, key=lambda k: strategy_b_results[k]['pr_auc'])
best_b_pr_auc = strategy_b_results[best_b_name]['pr_auc']

print(f"Strategy A (SMOTE 계열) 최고:  {best_a_name:20s} | PR-AUC = {best_a_pr_auc:.4f}")
print(f"Strategy B (Class Weight) 최고: {best_b_name:20s} | PR-AUC = {best_b_pr_auc:.4f}")
print("="*70)

if best_b_pr_auc > best_a_pr_auc:
    diff = best_b_pr_auc - best_a_pr_auc
    pct = diff / best_a_pr_auc * 100
    print(f"✅ 선택: Strategy B (Class Weight)")
    print(f"   이유: {diff:.4f} ({pct:.2f}%) 더 우수, 노이즈 증폭 위험 없음")
    selected_strategy = 'Class Weight'
    selected_weight = strategy_b_results[best_b_name]['weight']
else:
    diff = best_a_pr_auc - best_b_pr_auc
    pct = diff / best_b_pr_auc * 100
    print(f"✅ 선택: Strategy A (SMOTE 계열)")
    print(f"   이유: {diff:.4f} ({pct:.2f}%) 더 우수")
    selected_strategy = 'SMOTE'
    selected_weight = 1

print("="*70)

In [ ]:
# 시각화: 전략 비교
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('PR-AUC 비교', 'F2-Score 비교'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# PR-AUC
all_methods = list(strategy_a_results.keys()) + list(strategy_b_results.keys())
all_pr_aucs = [strategy_a_results[k]['pr_auc'] for k in strategy_a_results] + \
              [strategy_b_results[k]['pr_auc'] for k in strategy_b_results]
colors = ['#3498db']*len(strategy_a_results) + ['#2ecc71']*len(strategy_b_results)

fig.add_trace(
    go.Bar(x=all_methods, y=all_pr_aucs, marker_color=colors, showlegend=False,
           text=[f"{v:.4f}" for v in all_pr_aucs], textposition='auto'),
    row=1, col=1
)

# F2-Score
all_f2s = [strategy_a_results[k]['f2_score'] for k in strategy_a_results] + \
          [strategy_b_results[k]['f2_score'] for k in strategy_b_results]

fig.add_trace(
    go.Bar(x=all_methods, y=all_f2s, marker_color=colors, showlegend=False,
           text=[f"{v:.4f}" for v in all_f2s], textposition='auto'),
    row=1, col=2
)

fig.update_layout(
    title_text='리샘플링 전략 대조 실험 결과 (Validation Set)',
    height=500
)

fig.update_xaxes(tickangle=-45)

fig.show()

---

## ✅ Section 2-3 완료

**완료 사항:**
- ✅ 전처리 파이프라인 정의 (InfiniteHandler → Imputer → Scaler)
- ✅ 리샘플링 전략 대조 실험 (SMOTE vs Class Weight)
- ✅ Validation Set 기반 성능 비교
- ✅ 우수한 전략 선택

**다음 단계:**
- Section 4: AutoML 하이퍼파라미터 튜닝 (선택된 전략 적용)
- Section 5: 모델 선택 + 임계값 최적화

**⚠️ 중요 확인:**
- Test set은 여전히 사용하지 않음 ✅
- 모든 의사결정은 Validation set 기반 ✅

## 4. AutoML: 하이퍼파라미터 튜닝 ⭐

### 4.1 모델 및 파라미터 그리드 정의

**모델 선택:**
1. **LightGBM** - 빠르고 높은 정확도
2. **XGBoost** - 강력한 성능, 불균형 데이터에 효과적
3. **CatBoost** - 범주형 변수 처리 우수
4. **Logistic Regression** - 해석력, 다양성 확보
5. **Random Forest** - Tree 기반, 메커니즘 다름

**튜닝 전략:**
- RandomizedSearchCV (효율성)
- 5-Fold Stratified CV
- PR-AUC 메트릭 (불균형 데이터)
- Train Set만 사용 (Test/Validation 절대 미사용)

In [ ]:
# F2-Score Scorer 정의 (Recall 우선)
f2_scorer = make_scorer(fbeta_score, beta=2)
pr_auc_scorer = make_scorer(average_precision_score, needs_proba=True)

print("✅ Scorer 정의 완료")
print("   - PR-AUC: 주요 평가 지표 (불균형 데이터)")
print("   - F2-Score: Recall 우선 (beta=2)")

In [ ]:
# 선택된 전략에 따른 scale_pos_weight 설정
# (리샘플링 전략 실험 결과에 따라 자동 설정)
if selected_strategy == 'Class Weight':
    use_class_weight = True
    use_resampling = False
    print(f"✅ 선택된 전략: Class Weight (scale_pos_weight={selected_weight:.2f})")
else:
    use_class_weight = False
    use_resampling = True
    print(f"✅ 선택된 전략: SMOTE (sampling_strategy=0.2)")

### 4.2 파라미터 그리드

In [ ]:
# LightGBM 파라미터 그리드
lgbm_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10, -1],
    'num_leaves': [15, 31, 63, 127],
    'min_child_samples': [10, 20, 30, 50],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0, 0.01, 0.1, 1],
    'scale_pos_weight': [selected_weight] if use_class_weight else [1],
}

# XGBoost 파라미터 그리드
xgb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0, 0.01, 0.1, 1],
    'scale_pos_weight': [selected_weight] if use_class_weight else [1],
}

# CatBoost 파라미터 그리드
catboost_params = {
    'iterations': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [3, 5, 7, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'scale_pos_weight': [selected_weight] if use_class_weight else [1],
}

# Logistic Regression 파라미터 그리드
lr_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000],
    'class_weight': ['balanced'] if use_class_weight else [None],
}

# Random Forest 파라미터 그리드
rf_params = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': ['balanced'] if use_class_weight else [None],
}

print("✅ 파라미터 그리드 정의 완료")
print(f"\n각 모델별 탐색 공간:")
print(f"  - LightGBM: {np.prod([len(v) for v in lgbm_params.values()]):,} combinations")
print(f"  - XGBoost: {np.prod([len(v) for v in xgb_params.values()]):,} combinations")
print(f"  - CatBoost: {np.prod([len(v) for v in catboost_params.values()]):,} combinations")
print(f"  - Logistic Regression: {np.prod([len(v) for v in lr_params.values()]):,} combinations")
print(f"  - Random Forest: {np.prod([len(v) for v in rf_params.values()]):,} combinations")

### 4.3 모델별 RandomizedSearchCV

**⚠️ 주의: Train Set만 사용, Validation/Test는 절대 사용 안 함!**

In [ ]:
# 모델 정의
models = {
    'LightGBM': (
        lgb.LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1),
        lgbm_params,
        100  # n_iter
    ),
    'XGBoost': (
        xgb.XGBClassifier(random_state=RANDOM_STATE, n_jobs=-1, verbosity=0,
                         tree_method='hist', enable_categorical=False),
        xgb_params,
        100
    ),
    'CatBoost': (
        CatBoostClassifier(random_state=RANDOM_STATE, verbose=0),
        catboost_params,
        50  # CatBoost는 느리므로 50회
    ),
    'LogisticRegression': (
        LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1),
        lr_params,
        50
    ),
    'RandomForest': (
        RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
        rf_params,
        50
    )
}

print("✅ 모델 정의 완료")
print(f"\n튜닝할 모델: {list(models.keys())}")

In [ ]:
# RandomizedSearchCV 실행
print("🚀 AutoML 하이퍼파라미터 튜닝 시작...")
print("="*70)
print("⚠️  Train Set + 5-Fold CV만 사용 (Validation/Test 미사용)")
print("="*70)

search_results = {}
best_models = {}

for model_name, (model, param_grid, n_iter) in models.items():
    print(f"\n{'='*70}")
    print(f"📊 {model_name} 튜닝 중... (n_iter={n_iter})")
    print(f"{'='*70}")
    
    # RandomizedSearchCV
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=n_iter,
        scoring=pr_auc_scorer,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=1,
        return_train_score=True
    )
    
    # 학습 (Train set만 사용!)
    search.fit(X_train_processed, y_train)
    
    # 결과 저장
    search_results[model_name] = search
    best_models[model_name] = search.best_estimator_
    
    print(f"\n✅ {model_name} 완료")
    print(f"   - Best CV PR-AUC: {search.best_score_:.4f}")
    print(f"   - Best Params: {search.best_params_}")

print(f"\n{'='*70}")
print("✅ 모든 모델 튜닝 완료!")
print(f"{'='*70}")

### 4.4 결과 분석

In [ ]:
# 전체 결과 DataFrame 생성
all_results = []

for model_name, search in search_results.items():
    results_df = pd.DataFrame(search.cv_results_)
    results_df['model'] = model_name
    all_results.append(results_df)

all_results_df = pd.concat(all_results, ignore_index=True)

# 상위 10개 모델
top10 = all_results_df.nsmallest(10, 'rank_test_score')[[
    'model', 'mean_test_score', 'std_test_score', 'mean_train_score', 'params'
]].reset_index(drop=True)

top10.index = top10.index + 1

print("🏆 상위 10개 모델 (Train Set 5-Fold CV 기반)")
print("="*70)
print(top10.to_string())
print("="*70)

In [ ]:
# 모델별 최고 성능 비교
model_best_scores = {}
for model_name, search in search_results.items():
    model_best_scores[model_name] = search.best_score_

# 시각화
fig = go.Figure()

models_sorted = sorted(model_best_scores.items(), key=lambda x: x[1], reverse=True)
model_names = [m[0] for m in models_sorted]
scores = [m[1] for m in models_sorted]

fig.add_trace(go.Bar(
    x=model_names,
    y=scores,
    text=[f"{s:.4f}" for s in scores],
    textposition='auto',
    marker_color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']
))

fig.update_layout(
    title='모델별 최고 성능 비교 (Train Set 5-Fold CV PR-AUC)',
    xaxis_title='모델',
    yaxis_title='PR-AUC',
    height=500
)

fig.show()

print(f"\n🥇 Best Model: {model_names[0]} (PR-AUC: {scores[0]:.4f})")

In [ ]:
# 과적합 체크 (Train vs CV Score)
print("\n📊 과적합 체크 (Train Score - CV Score)")
print("="*70)

for model_name, search in search_results.items():
    train_score = search.cv_results_['mean_train_score'][search.best_index_]
    cv_score = search.best_score_
    gap = train_score - cv_score
    
    status = "⚠️ 과적합 우려" if gap > 0.1 else "✅ 정상"
    
    print(f"{model_name:20s} | Train: {train_score:.4f} | CV: {cv_score:.4f} | "
          f"Gap: {gap:.4f} | {status}")

print("="*70)

---

## ✅ Section 4 완료

**완료 사항:**
- ✅ 5개 모델 파라미터 그리드 정의
- ✅ RandomizedSearchCV (Train Set + 5-Fold CV만)
- ✅ 상위 10개 모델 도출
- ✅ 과적합 체크

**다음 단계:**
- Section 5: 모델 선택 (Validation Set 기반 + Statistical Test)
- Section 6: 임계값 최적화 + Traffic Light (Validation Set)
- Section 7: Test Set 최종 평가 (단 한 번만!)

**⚠️ 중요 확인:**
- Test set은 여전히 사용하지 않음 ✅
- Validation set도 아직 사용하지 않음 ✅
- 모든 튜닝은 Train Set 5-Fold CV로만 진행 ✅

## 5. 모델 선택: Validation Set 기반 ⭐⭐⭐

### 5.1 Single Best Model vs Voting Ensemble

**중요:** Test set은 절대 사용하지 않고, **Validation set**으로만 모델을 선택합니다.

In [ ]:
# Single Best Model (CV 기준)
best_model_name = max(model_best_scores, key=model_best_scores.get)
best_model = best_models[best_model_name]

print(f"🥇 Single Best Model (Train CV 기준): {best_model_name}")
print(f"   - Train CV PR-AUC: {model_best_scores[best_model_name]:.4f}")

# Validation Set 평가
y_val_prob_single = best_model.predict_proba(X_val_processed)[:, 1]
val_pr_auc_single = average_precision_score(y_val, y_val_prob_single)

print(f"   - Validation PR-AUC: {val_pr_auc_single:.4f}")
print(f"   - 일반화 성능: {val_pr_auc_single - model_best_scores[best_model_name]:.4f}")

In [ ]:
# Voting Ensemble (상위 3개 모델)
top3_models = sorted(model_best_scores.items(), key=lambda x: x[1], reverse=True)[:3]

print(f"\n🏆 Top 3 모델 (Voting Ensemble 후보):")
for i, (name, score) in enumerate(top3_models, 1):
    print(f"   {i}. {name}: {score:.4f}")

# Voting Classifier 생성
voting_estimators = [(name, best_models[name]) for name, _ in top3_models]

voting_clf = VotingClassifier(
    estimators=voting_estimators,
    voting='soft',
    n_jobs=-1
)

# Train set으로 학습
print(f"\n🔄 Voting Ensemble 학습 중...")
voting_clf.fit(X_train_processed, y_train)

# Validation Set 평가
y_val_prob_voting = voting_clf.predict_proba(X_val_processed)[:, 1]
val_pr_auc_voting = average_precision_score(y_val, y_val_prob_voting)

print(f"✅ Voting Ensemble 완료")
print(f"   - Validation PR-AUC: {val_pr_auc_voting:.4f}")

In [ ]:
# Statistical Significance Test (Wilcoxon)
print(f"\n📊 Statistical Significance Test (Wilcoxon)")
print("="*70)

# Train set에서 CV fold별 점수 비교
single_cv_scores = cross_val_score(
    best_model, X_train_processed, y_train,
    cv=5, scoring=pr_auc_scorer, n_jobs=-1
)

voting_cv_scores = cross_val_score(
    voting_clf, X_train_processed, y_train,
    cv=5, scoring=pr_auc_scorer, n_jobs=-1
)

# Wilcoxon test
try:
    statistic, pvalue = wilcoxon(voting_cv_scores, single_cv_scores)
    print(f"Single Model CV Scores: {single_cv_scores}")
    print(f"Voting Model CV Scores: {voting_cv_scores}")
    print(f"\nWilcoxon p-value: {pvalue:.4f}")
    
    if pvalue < 0.05:
        print(f"   ✅ 유의미한 차이 있음 (p < 0.05)")
    else:
        print(f"   ⚠️ 유의미한 차이 없음 (p >= 0.05)")
except:
    print("   ⚠️ Wilcoxon test 실패 (점수 동일 가능성)")
    pvalue = 1.0

print("="*70)

In [ ]:
# 최종 모델 선택 (Validation 기반)
print(f"\n🎯 최종 모델 선택 (Validation Set 기반)")
print("="*70)
print(f"Single Best Model:     PR-AUC = {val_pr_auc_single:.4f}")
print(f"Voting Ensemble:       PR-AUC = {val_pr_auc_voting:.4f}")
print("="*70)

# 선택 로직
improvement_threshold = 0.005  # 0.5% 이상 차이 나야 의미 있음
improvement = val_pr_auc_voting - val_pr_auc_single

if improvement > improvement_threshold and pvalue < 0.1:
    final_model = voting_clf
    final_model_name = 'VotingEnsemble'
    final_val_pr_auc = val_pr_auc_voting
    decision_reason = f"Voting이 {improvement:.4f} 더 우수 (p={pvalue:.4f} < 0.1)"
    print(f"✅ 선택: Voting Ensemble")
    print(f"   이유: {decision_reason}")
else:
    final_model = best_model
    final_model_name = best_model_name
    final_val_pr_auc = val_pr_auc_single
    if improvement <= improvement_threshold:
        decision_reason = f"성능 차이 미미 ({improvement:.4f} < {improvement_threshold}) + SHAP 해석력 우선"
    else:
        decision_reason = f"통계적 유의성 부족 (p={pvalue:.4f} >= 0.1)"
    print(f"✅ 선택: Single Model ({best_model_name})")
    print(f"   이유: {decision_reason}")

print("="*70)

# 최종 모델 정보 저장
model_selection_info = {
    'final_model_name': final_model_name,
    'val_pr_auc': final_val_pr_auc,
    'decision_reason': decision_reason,
    'single_model_name': best_model_name,
    'single_val_pr_auc': val_pr_auc_single,
    'voting_val_pr_auc': val_pr_auc_voting,
    'wilcoxon_pvalue': pvalue
}

## 6. 임계값 최적화: Validation Set 기반 ⭐⭐⭐

### 6.1 F2-Score 최적 임계값

**중요:** Validation set으로만 임계값을 결정하고, Test set에는 적용만 합니다.

In [ ]:
# Validation set 확률 예측
y_val_prob = final_model.predict_proba(X_val_processed)[:, 1]

# Precision-Recall Curve
precisions_val, recalls_val, thresholds_val = precision_recall_curve(y_val, y_val_prob)

# F2-Score 계산 (Recall 우선)
beta = 2
f2_scores = (1 + beta**2) * (precisions_val[:-1] * recalls_val[:-1]) / \
            (beta**2 * precisions_val[:-1] + recalls_val[:-1] + 1e-10)

# 최적 임계값
optimal_idx = np.argmax(f2_scores)
optimal_threshold_f2 = thresholds_val[optimal_idx]
optimal_f2 = f2_scores[optimal_idx]
optimal_precision = precisions_val[optimal_idx]
optimal_recall = recalls_val[optimal_idx]

print(f"📊 F2-Score 최적 임계값 (Validation Set 기반)")
print("="*70)
print(f"최적 임계값:  {optimal_threshold_f2:.4f}")
print(f"F2-Score:     {optimal_f2:.4f}")
print(f"Precision:    {optimal_precision:.2%}")
print(f"Recall:       {optimal_recall:.2%}")
print("="*70)

In [ ]:
# Recall 목표 기반 임계값
target_recall = 0.80  # 80% Recall 목표

# Recall >= 80% 인 임계값 중 Precision 최대
idx_recall_80 = np.where(recalls_val[:-1] >= target_recall)[0]

if len(idx_recall_80) > 0:
    best_idx = idx_recall_80[np.argmax(precisions_val[idx_recall_80])]
    threshold_recall_80 = thresholds_val[best_idx]
    recall_80_precision = precisions_val[best_idx]
    recall_80_recall = recalls_val[best_idx]
    
    print(f"\n📊 Recall 80% 보장 임계값 (Validation Set 기반)")
    print("="*70)
    print(f"임계값:       {threshold_recall_80:.4f}")
    print(f"Precision:    {recall_80_precision:.2%}")
    print(f"Recall:       {recall_80_recall:.2%}")
    print("="*70)
else:
    print(f"\n⚠️ Recall {target_recall:.0%} 달성 불가능")
    threshold_recall_80 = optimal_threshold_f2
    recall_80_precision = optimal_precision
    recall_80_recall = optimal_recall

In [ ]:
# 비즈니스 요구사항에 따라 선택 (Recall 우선)
selected_threshold = threshold_recall_80  # Recall 80% 보장 선택

print(f"\n✅ 최종 선택 임계값: {selected_threshold:.4f}")
print(f"   이유: Recall {target_recall:.0%} 보장 (부도 미탐지 최소화)")

In [ ]:
# PR Curve 시각화 (Validation Set)
fig = go.Figure()

# PR Curve
fig.add_trace(go.Scatter(
    x=recalls_val,
    y=precisions_val,
    mode='lines',
    name=f'PR Curve (AUC={final_val_pr_auc:.4f})',
    line=dict(color='blue', width=2)
))

# 최적 임계값 표시
fig.add_trace(go.Scatter(
    x=[recall_80_recall],
    y=[recall_80_precision],
    mode='markers',
    name=f'선택 임계값 ({selected_threshold:.4f})',
    marker=dict(size=15, color='red', symbol='star')
))

# Baseline
baseline = y_val.mean()
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[baseline, baseline],
    mode='lines',
    name=f'Baseline ({baseline:.2%})',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title='Precision-Recall Curve (Validation Set)',
    xaxis_title='Recall',
    yaxis_title='Precision',
    height=500
)

fig.show()

## 7. Traffic Light 시스템: 데이터 기반 임계값 ⭐

### 7.1 임계값 설계 (Validation Set 기반)

**논리:**
- 🔴 Red: Recall 80% 보장 임계값
- 🟡 Yellow: Recall 95% 보장 임계값
- 🟢 Green: Red 미만

In [ ]:
# Red 임계값 (Recall 80%)
red_threshold = selected_threshold

# Yellow 임계값 (Recall 95%)
target_recall_95 = 0.95
idx_recall_95 = np.where(recalls_val[:-1] >= target_recall_95)[0]

if len(idx_recall_95) > 0:
    best_idx_95 = idx_recall_95[np.argmax(precisions_val[idx_recall_95])]
    yellow_threshold = thresholds_val[best_idx_95]
    yellow_precision = precisions_val[best_idx_95]
    yellow_recall = recalls_val[best_idx_95]
else:
    # Recall 95% 불가능하면 최소 임계값 사용
    yellow_threshold = np.percentile(y_val_prob, 5)  # 하위 5%
    yellow_idx = np.argmin(np.abs(thresholds_val - yellow_threshold))
    yellow_precision = precisions_val[yellow_idx]
    yellow_recall = recalls_val[yellow_idx]

print(f"🚦 Traffic Light 임계값 (Validation Set 기반)")
print("="*70)
print(f"🔴 Red Threshold:    {red_threshold:.4f}")
print(f"   - 목표: Recall {target_recall:.0%} 보장")
print(f"   - Precision: {recall_80_precision:.2%}, Recall: {recall_80_recall:.2%}")
print(f"\n🟡 Yellow Threshold: {yellow_threshold:.4f}")
print(f"   - 목표: Recall {target_recall_95:.0%} 보장")
print(f"   - Precision: {yellow_precision:.2%}, Recall: {yellow_recall:.2%}")
print(f"\n🟢 Green:            < {yellow_threshold:.4f}")
print("="*70)

In [ ]:
# Validation Set에서 Traffic Light 성능 검증
def assign_traffic_light(prob, red_th, yellow_th):
    if prob >= red_th:
        return 'Red'
    elif prob >= yellow_th:
        return 'Yellow'
    else:
        return 'Green'

val_traffic_light = [assign_traffic_light(p, red_threshold, yellow_threshold) for p in y_val_prob]
val_df = pd.DataFrame({
    'actual': y_val.values,
    'prob': y_val_prob,
    'traffic_light': val_traffic_light
})

# 등급별 통계
traffic_stats = val_df.groupby('traffic_light').agg({
    'actual': ['count', 'sum', 'mean']
}).round(4)

traffic_stats.columns = ['기업 수', '실제 부도', '부도율']
traffic_stats = traffic_stats.reindex(['Red', 'Yellow', 'Green'])

print(f"\n📊 Traffic Light 성능 (Validation Set)")
print("="*70)
print(traffic_stats)
print("="*70)

# 포착률 계산
total_bankrupts = y_val.sum()
red_yellow_bankrupts = val_df[val_df['traffic_light'].isin(['Red', 'Yellow'])]['actual'].sum()
capture_rate = red_yellow_bankrupts / total_bankrupts

print(f"\n✅ 리스크 방어율 (Red+Yellow): {capture_rate:.2%}")
print(f"   - 전체 부도 {total_bankrupts}개 중 {red_yellow_bankrupts}개 포착")

---

## 8. Test Set 최종 평가 ⭐⭐⭐

### 8.1 중요 경고

```
⚠️⚠️⚠️ Test Set 사용 경고 ⚠️⚠️⚠️

이 섹션은 노트북에서 단 한 번만 실행됩니다.
Test Set은 최종 보고를 위한 평가 목적으로만 사용되며,
이전 모든 의사결정(모델 선택, 임계값 등)은
Validation Set에서만 이루어졌음을 보장합니다.

Test Set을 보고 다시 튜닝하거나 변경하지 않습니다!
```

In [ ]:
# Test Set 전처리 (Train set에 fit한 preprocessor 사용)
X_test_processed = preprocessor.transform(X_test)
X_test_processed = pd.DataFrame(X_test_processed, columns=X_test.columns, index=X_test.index)

# Test Set 예측
y_test_prob = final_model.predict_proba(X_test_processed)[:, 1]
y_test_pred = (y_test_prob >= selected_threshold).astype(int)

print(f"✅ Test Set 예측 완료")
print(f"   - 임계값: {selected_threshold:.4f} (Validation에서 결정됨)")

In [ ]:
# Test Set 최종 평가
test_pr_auc = average_precision_score(y_test, y_test_prob)
test_roc_auc = roc_auc_score(y_test, y_test_prob)

# F2-Score, Precision, Recall
test_f2 = fbeta_score(y_test, y_test_pred, beta=2)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

# Type II Error
type_ii_error = fn / (fn + tp) if (fn + tp) > 0 else 0

print(f"\n{'='*70}")
print(f"🎯 Test Set 최종 평가 (임계값: {selected_threshold:.4f})")
print(f"{'='*70}")
print(f"⚠️  이 결과는 최종 보고용이며, 이전 단계에서 Test Set을")
print(f"   절대 사용하지 않았음을 보장합니다.")
print(f"{'='*70}")
print(f"\n📊 성능 메트릭:")
print(f"   PR-AUC:              {test_pr_auc:.4f}")
print(f"   ROC-AUC:             {test_roc_auc:.4f}")
print(f"   F2-Score:            {test_f2:.4f}")
print(f"   Precision:           {test_precision:.2%}")
print(f"   Recall:              {test_recall:.2%}")
print(f"   Type II Error:       {type_ii_error:.2%}")
print(f"\n📊 Confusion Matrix:")
print(f"   TN: {tn:,}  |  FP: {fp:,}")
print(f"   FN: {fn:,}    |  TP: {tp:,}")
print(f"{'='*70}")

# Validation vs Test 비교
print(f"\n📊 Validation vs Test 비교 (일반화 성능 검증):")
print(f"   Validation PR-AUC: {final_val_pr_auc:.4f}")
print(f"   Test PR-AUC:       {test_pr_auc:.4f}")
print(f"   차이:              {test_pr_auc - final_val_pr_auc:.4f}")

if abs(test_pr_auc - final_val_pr_auc) < 0.02:
    print(f"   ✅ 일반화 성능 우수 (차이 < 2%)")
else:
    print(f"   ⚠️  일반화 성능 주의 (차이 >= 2%)")

In [ ]:
# Test Set Traffic Light
test_traffic_light = [assign_traffic_light(p, red_threshold, yellow_threshold) for p in y_test_prob]
test_df = pd.DataFrame({
    'actual': y_test.values,
    'prob': y_test_prob,
    'traffic_light': test_traffic_light
})

# 등급별 통계
test_traffic_stats = test_df.groupby('traffic_light').agg({
    'actual': ['count', 'sum', 'mean']
}).round(4)

test_traffic_stats.columns = ['기업 수', '실제 부도', '부도율']
test_traffic_stats = test_traffic_stats.reindex(['Red', 'Yellow', 'Green'])

print(f"\n🚦 Traffic Light 시스템 (Test Set 최종 평가)")
print("="*70)
print(test_traffic_stats)
print("="*70)

# 포착률
test_total_bankrupts = y_test.sum()
test_red_yellow_bankrupts = test_df[test_df['traffic_light'].isin(['Red', 'Yellow'])]['actual'].sum()
test_capture_rate = test_red_yellow_bankrupts / test_total_bankrupts

print(f"\n✅ 리스크 방어율 (Red+Yellow): {test_capture_rate:.2%}")
print(f"   - 전체 부도 {test_total_bankrupts}개 중 {test_red_yellow_bankrupts}개 포착")

In [ ]:
# PR Curve & ROC Curve (Test Set)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Precision-Recall Curve', 'ROC Curve')
)

# PR Curve
precisions_test, recalls_test, _ = precision_recall_curve(y_test, y_test_prob)
fig.add_trace(
    go.Scatter(x=recalls_test, y=precisions_test, mode='lines', name=f'PR (AUC={test_pr_auc:.4f})'),
    row=1, col=1
)
baseline_test = y_test.mean()
fig.add_trace(
    go.Scatter(x=[0, 1], y=[baseline_test, baseline_test], mode='lines', 
               name='Baseline', line=dict(dash='dash')),
    row=1, col=1
)

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_test_prob)
fig.add_trace(
    go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC (AUC={test_roc_auc:.4f})'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', 
               line=dict(dash='dash'), showlegend=False),
    row=1, col=2
)

fig.update_xaxes(title_text='Recall', row=1, col=1)
fig.update_xaxes(title_text='FPR', row=1, col=2)
fig.update_yaxes(title_text='Precision', row=1, col=1)
fig.update_yaxes(title_text='TPR', row=1, col=2)

fig.update_layout(title_text='Test Set 성능 곡선', height=500)
fig.show()

## 9. Feature Importance 분석

### 9.1 모델 Feature Importance

In [ ]:
# Feature Importance 추출
if final_model_name == 'VotingEnsemble':
    # Voting의 경우 첫 번째 estimator 사용
    model_for_importance = final_model.estimators_[0]
    print(f"⚠️  Voting Ensemble이므로 첫 번째 모델({top3_models[0][0]})의 Feature Importance 사용")
else:
    model_for_importance = final_model

# Feature Importance 가져오기
if hasattr(model_for_importance, 'feature_importances_'):
    importances = model_for_importance.feature_importances_
elif hasattr(model_for_importance, 'coef_'):
    importances = np.abs(model_for_importance.coef_[0])
else:
    print("⚠️  Feature Importance를 추출할 수 없습니다.")
    importances = np.zeros(len(X_train.columns))

# DataFrame 생성
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': importances
}).sort_values('importance', ascending=False)

print(f"\n📊 Feature Importance Top 15")
print("="*70)
print(feature_importance_df.head(15).to_string(index=False))
print("="*70)

In [ ]:
# Feature Importance 시각화
fig = go.Figure()

top15 = feature_importance_df.head(15)

fig.add_trace(go.Bar(
    x=top15['importance'][::-1],
    y=top15['feature'][::-1],
    orientation='h',
    marker_color='#3498db',
    text=[f"{v:.4f}" for v in top15['importance'][::-1]],
    textposition='auto'
))

fig.update_layout(
    title=f'Feature Importance Top 15 ({final_model_name})',
    xaxis_title='Importance',
    yaxis_title='Feature',
    height=600
)

fig.show()

### 9.2 비즈니스 임팩트 분석: Cumulative Gains Curve

In [ ]:
# Cumulative Gains Curve (Test Set)
# 확률 순으로 정렬
test_sorted_idx = np.argsort(y_test_prob)[::-1]
y_test_sorted = y_test.values[test_sorted_idx]

# 누적 부도 포착률
cumulative_bankrupts = np.cumsum(y_test_sorted)
total_bankrupts = y_test.sum()
cumulative_pct = cumulative_bankrupts / total_bankrupts * 100

# 검토 비율 (상위 몇 %)
review_pct = np.arange(1, len(y_test) + 1) / len(y_test) * 100

# 시각화
fig = go.Figure()

# Cumulative Gains
fig.add_trace(go.Scatter(
    x=review_pct,
    y=cumulative_pct,
    mode='lines',
    name='모델 성능',
    line=dict(color='blue', width=2)
))

# Random Baseline
fig.add_trace(go.Scatter(
    x=[0, 100],
    y=[0, 100],
    mode='lines',
    name='Random',
    line=dict(color='gray', dash='dash')
))

# 주요 지점 표시
for review_threshold in [10, 20, 30]:
    idx = int(len(y_test) * review_threshold / 100)
    gain = cumulative_pct[idx-1]
    fig.add_annotation(
        x=review_threshold,
        y=gain,
        text=f"상위 {review_threshold}%: {gain:.1f}% 포착",
        showarrow=True,
        arrowhead=2
    )

fig.update_layout(
    title='Cumulative Gains Curve (Test Set)',
    xaxis_title='검토한 기업 비율 (%)',
    yaxis_title='포착한 부도 기업 비율 (%)',
    height=500
)

fig.show()

# 효율성 분석
print(f"\n💼 비즈니스 임팩트 분석")
print("="*70)
print(f"상위 10% 검토 시: {cumulative_pct[int(len(y_test)*0.1)-1]:.1f}% 부도 포착")
print(f"상위 20% 검토 시: {cumulative_pct[int(len(y_test)*0.2)-1]:.1f}% 부도 포착")
print(f"상위 30% 검토 시: {cumulative_pct[int(len(y_test)*0.3)-1]:.1f}% 부도 포착")
print("="*70)
print(f"\n💡 해석: 상위 20% 기업만 검토해도 부도 기업의 약 {cumulative_pct[int(len(y_test)*0.2)-1]:.0f}%를 사전 포착 가능")

## 10. 모델 저장 및 다음 단계

### 10.1 모델 및 데이터 저장 (Part 4 SHAP 분석용)

In [ ]:
# 저장 경로
model_prefix = 'Part3_v2'

# 1. 최종 모델
final_model_path = os.path.join(PROCESSED_DIR, f'{model_prefix}_최종모델.pkl')
joblib.dump(final_model, final_model_path)
print(f"✅ 최종 모델 저장: {final_model_path}")

# 2. 전처리 파이프라인
preprocessor_path = os.path.join(PROCESSED_DIR, f'{model_prefix}_전처리기.pkl')
joblib.dump(preprocessor, preprocessor_path)
print(f"✅ 전처리기 저장: {preprocessor_path}")

# 3. 전처리된 데이터 (SHAP 분석용)
X_train_processed.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_X_train_processed.csv'), 
                         index=False, encoding='utf-8-sig')
X_val_processed.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_X_val_processed.csv'), 
                       index=False, encoding='utf-8-sig')
X_test_processed.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_X_test_processed.csv'), 
                        index=False, encoding='utf-8-sig')
print(f"✅ 전처리된 데이터 저장 (Train/Val/Test)")

# 4. 타겟 변수
y_train.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_y_train.csv'), 
               index=False, encoding='utf-8-sig', header=['target'])
y_val.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_y_val.csv'), 
             index=False, encoding='utf-8-sig', header=['target'])
y_test.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_y_test.csv'), 
              index=False, encoding='utf-8-sig', header=['target'])
print(f"✅ 타겟 변수 저장 (Train/Val/Test)")

# 5. 임계값 및 메타데이터
metadata = {
    'final_model_name': final_model_name,
    'selected_threshold': selected_threshold,
    'red_threshold': red_threshold,
    'yellow_threshold': yellow_threshold,
    'val_pr_auc': final_val_pr_auc,
    'test_pr_auc': test_pr_auc,
    'test_f2': test_f2,
    'test_precision': test_precision,
    'test_recall': test_recall,
    'test_type_ii_error': type_ii_error,
    'model_selection_info': model_selection_info,
    'feature_names': list(X_train.columns),
    'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

metadata_path = os.path.join(PROCESSED_DIR, f'{model_prefix}_메타데이터.pkl')
joblib.dump(metadata, metadata_path)
print(f"✅ 메타데이터 저장: {metadata_path}")

# 6. Feature Importance
feature_importance_df.to_csv(os.path.join(PROCESSED_DIR, f'{model_prefix}_feature_importance.csv'), 
                             index=False, encoding='utf-8-sig')
print(f"✅ Feature Importance 저장")

In [ ]:
# 저장된 파일 목록
print(f"\n📁 저장된 파일 목록:")
print("="*70)
import glob
saved_files = glob.glob(os.path.join(PROCESSED_DIR, f'{model_prefix}*'))
for f in sorted(saved_files):
    size = os.path.getsize(f) / 1024
    print(f"  - {os.path.basename(f)} ({size:.1f} KB)")
print("="*70)

### 10.2 최종 요약

In [ ]:
print(f"\n{'='*70}")
print(f"🎉 Part 3 v2 모델링 및 최적화 완료!")
print(f"{'='*70}")
print(f"\n📊 최종 결과 요약:")
print(f"\n1. 데이터 분할:")
print(f"   - Train Set:      {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)")
print(f"   - Validation Set: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%)")
print(f"   - Test Set:       {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)")

print(f"\n2. 리샘플링 전략:")
print(f"   - 선택: {selected_strategy}")

print(f"\n3. 최종 모델:")
print(f"   - 모델: {final_model_name}")
print(f"   - Validation PR-AUC: {final_val_pr_auc:.4f}")
print(f"   - Test PR-AUC:       {test_pr_auc:.4f}")
print(f"   - 선택 이유: {decision_reason}")

print(f"\n4. 임계값:")
print(f"   - 최적 임계값: {selected_threshold:.4f}")
print(f"   - Red Threshold:    {red_threshold:.4f} (Recall 80% 보장)")
print(f"   - Yellow Threshold: {yellow_threshold:.4f} (Recall 95% 보장)")

print(f"\n5. Test Set 최종 성능:")
print(f"   - PR-AUC:        {test_pr_auc:.4f}")
print(f"   - ROC-AUC:       {test_roc_auc:.4f}")
print(f"   - F2-Score:      {test_f2:.4f}")
print(f"   - Precision:     {test_precision:.2%}")
print(f"   - Recall:        {test_recall:.2%}")
print(f"   - Type II Error: {type_ii_error:.2%}")

print(f"\n6. Traffic Light 성능 (Test Set):")
print(f"   - 리스크 방어율: {test_capture_rate:.2%}")
print(f"   - Red 기업 부도율:    {test_traffic_stats.loc['Red', '부도율']*100:.2f}%")
print(f"   - Yellow 기업 부도율: {test_traffic_stats.loc['Yellow', '부도율']*100:.2f}%")
print(f"   - Green 기업 부도율:  {test_traffic_stats.loc['Green', '부도율']*100:.2f}%")

print(f"\n7. Data Leakage 방지 확인:")
print(f"   ✅ Test set은 최종 평가에서만 단 한 번 사용")
print(f"   ✅ 모든 의사결정(모델 선택, 임계값)은 Validation set 기반")
print(f"   ✅ 하이퍼파라미터 튜닝은 Train set 5-Fold CV만 사용")

print(f"\n{'='*70}")
print(f"\n📌 다음 단계: Part 4 - SHAP 분석 및 모델 해석")
print(f"{'='*70}")

---

## ✅ Part 3 v2 완료

**핵심 개선 사항:**
1. ✅ **Data Leakage 완전 제거** - Test set은 최종 평가만
2. ✅ **3-Way Split** - Train/Validation/Test 명확한 분리
3. ✅ **리샘플링 전략 대조 실험** - SMOTE vs Class Weight
4. ✅ **Validation 기반 모델 선택** - Statistical Test 포함
5. ✅ **Validation 기반 임계값 최적화** - F2-Score, Recall 우선
6. ✅ **데이터 기반 Traffic Light** - Recall 보장 임계값
7. ✅ **Test Set 단 한 번 평가** - 일반화 성능 검증

**학술적 엄밀성:**
- ✅ 재현 가능성 (random_state 설정)
- ✅ Stratified Split (부도율 유지)
- ✅ Cross-Validation (5-Fold)
- ✅ Statistical Significance Test (Wilcoxon)

**비즈니스 가치:**
- ✅ F2-Score (Recall 우선)
- ✅ Traffic Light 시스템 (의사결정 지원)
- ✅ Cumulative Gains (효율성 입증)
- ✅ Type II Error 최소화

**Part 4 준비:**
- ✅ 모델 저장 완료
- ✅ 전처리 파이프라인 저장
- ✅ 전처리된 데이터 저장 (SHAP 분석용)
- ✅ Feature Importance 저장